In [2]:
import cv2
import numpy as np
import sys

### 動画の保存

In [6]:
cap = cv2.VideoCapture(0)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
codec = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter('output.mp4', codec, fps, (w, h))
while True:
    ret, frame = cap.read()
    video.write(frame)
    if not ret:
        break
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cap.release()
cv2.destroyAllWindows()

### 動画の再生

In [4]:
cap = cv2.VideoCapture('output.mp4')

while True:
    ret, frame = cap.read()
    if not ret:
        break
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

### グレースケール変換

In [5]:
cap = cv2.VideoCapture('output.mp4')

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

### Canny法

In [6]:
cap = cv2.VideoCapture('output.mp4')

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.Canny(frame, 40, 200)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

### フレーム分割

In [5]:
cap = cv2.VideoCapture('output.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)

for n in range(100):
    cap.set(cv2.CAP_PROP_POS_FRAMES, n)
    ret, frame = cap.read()
    if ret:
        cv2.imwrite(f'frames/frame{n}.png', frame)
    else :
        print('Error reading frame')
        break

    

### 背景差分

In [6]:
bg = cv2.imread('frames/frame0.png', cv2.IMREAD_GRAYSCALE)
frame = cv2.imread('frames/frame91.png', cv2.IMREAD_GRAYSCALE)
diff = cv2.absdiff(frame, bg)
cv2.imshow('diff', diff)
_, mask = cv2.threshold(diff, 40, 255, cv2.THRESH_BINARY)
kernel = np.ones((3, 3), np.uint8)
mask = cv2.dilate(mask, kernel, iterations=2)
mask = cv2.erode(mask, kernel, iterations=2)
cv2.imshow('mask', mask)
dst = cv2.bitwise_and(frame, frame, mask=mask)
cv2.imshow('dst', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

### フレーム間差分

In [3]:
I1 = cv2.imread('frames/frame90.png', cv2.IMREAD_GRAYSCALE)
I2 = cv2.imread('frames/frame91.png', cv2.IMREAD_GRAYSCALE)
I3 = cv2.imread('frames/frame92.png', cv2.IMREAD_GRAYSCALE)
img_diff1 = cv2.absdiff(I1, I2)
img_diff2 = cv2.absdiff(I2, I3)
cv2.imshow('diff1', img_diff1)
cv2.imshow('diff2', img_diff2)
img_and = cv2.bitwise_and(img_diff1, img_diff2)
_, img_th = cv2.threshold(img_and, 40, 255, cv2.THRESH_BINARY)
kernel = np.ones((3, 3), np.uint8)
img_dialate = cv2.dilate(img_th, kernel, iterations=2)
img_mask = cv2.erode(img_dialate, kernel, iterations=2)
cv2.imshow('mask', img_mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 動体検知

In [3]:
cap = cv2.VideoCapture(0)
ret, bg = cap.read()
bg_gray = cv2.cvtColor(bg, cv2.COLOR_BGR2GRAY)
while True:
    ret, frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    diff = cv2.absdiff(frame_gray, bg_gray)
    _, mask = cv2.threshold(diff, 40, 255, cv2.THRESH_BINARY)
    element4 = np.array([[0, 1, 0], [1, 1, 1], [0, 1, 0]], np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, element4)
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if cv2.contourArea(contour) > 1000:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            frame = cv2.putText(frame, 'Motion Detected', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.imshow('frame', frame)
    cv2.imshow('mask', mask)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()

## 機械学習

In [1]:
%reset -f
import cv2
from ultralytics import YOLO

### 顔検出

In [2]:
import cv2
cap = cv2.VideoCapture(0)
cascade = cv2.CascadeClassifier('cascade/haarcascade_frontalface_alt.xml')
while True:
    ret, frame = cap.read()
    if ret:
        face = cascade.detectMultiScale(frame)
        if len(face):
            for x, y, w, h in face:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) == ord('q'):
            break
    else :
        break

cap.release()
cv2.destroyAllWindows()

### 物体追跡

In [3]:
model = YOLO('cascade/yolov8n.pt')
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if ret:
        results = model.track(frame)
        frame_track = results[0].plot()
        cv2.imshow('frame', frame_track)
        if cv2.waitKey(1) == ord('q'):
            break
    else :
        break

cap.release()
cv2.destroyAllWindows()

100%|██████████| 6.25M/6.25M [00:02<00:00, 2.91MB/s]


requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
  Obtaining dependency information for lapx>=0.5.2 from https://files.pythonhosted.org/packages/f3/2b/6c329de5ca45759d3c19d571fcd406b4af108e7c7aaa84e34ab052e31d8b/lapx-0.5.11-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 1.5/1.5 MB 2.9 MB/s eta 0:00:00

requirements: AutoUpdate success  5.8s, installed 1 package: ['lapx>=0.5.2']
requirements:  Restart runtime or rerun command for updates to take effect


0: 480x640 1 person, 1 chair, 114.6ms
Speed: 4.9ms preprocess, 114.6ms inference, 7.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 chair, 64.0ms
Speed: 2.0ms preprocess, 64.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 chair, 65.8ms
Speed: 1.0ms preprocess, 65.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 chair, 56.9ms
Speed: 2.0ms preprocess, 